In [ ]:
# INSTRUCTIONS:
# Click on run cell button on every single cell
# After running this cell
# CLICK THE URL PROVIDED BY THE CODE OUTPUT, PASTE THE STRING  TO THE TEXTBOX AND PRESS ENTER
# ---------------------------------------------------------------------------------------------


# You can change some variables in selected cells so that you can generate custom reports.
# Skip this if you don't want to know how this is set up this way
# Since Colab runs on a range of IPs whenever we connect to runtimes, we are only to connect to our Cloud SQL database by using proxy
# We apt install screen because when we run the proxy starts, it has to stay active to listen to incoming connections, the other cells in the book can't start running. 
# So the screen will keep our proxy running in the background, and we can connect to our Cloud SQL to retrieve data.


# Import all modules required and install missing ones
!pip install pandas
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from operator import attrgetter
import matplotlib.colors as mcolors
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy --quiet
!mkdir -p /cloudsql
!chmod +x cloud_sql_proxy --quiet
!pip install git+https://github.com/GoogleCloudPlatform/cloud-sql-python-connector --quiet

from google.cloud.sql.connector import connector
!apt -qq update && apt -qq install  -y   screen


# Authenticate your connection by your work email
# CLICK THE URL PROVIDED BY THE CODE OUTPUT, PASTE THE STRING  TO THE TEXTBOX AND PRESS ENTER
from google.colab import auth
auth.authenticate_user()


In [ ]:

# Connect to Cloud SQL proxy and let it run in the background with screen
!screen -S test -d -m "./projectid:region:dbname--dir /cloudsql"
from pymysql import*
import xlwt
import pandas.io.sql as sql

# Connection settings, you can change the db instance here
con = connector.connect(
    "#projectid:region:dbname", 
    "pymysql",
    user="",
    password="",
    db=""
)

# This is the actually query that will be run. You can change the created_at timeframe or order_status_label, etc
raw_data =sql.read_sql("SELECT user_id, payment_at, grand_total, id FROM ### WHERE ####",con)
# have a sneak peek at the raw data


In [ ]:
# Convert data type of a column to float
raw_data['grand_total'] = raw_data['grand_total'].astype(float)


# Choose a dataframe that we want to make cohort analysis
# df = with_phone_number.copy()

df_users = raw_data.copy()
df_orders = raw_data.copy()
df_gmv = raw_data.copy()




In [ ]:

# Create cohort_pivot for df_users to get number of unique customers retentions
# Create order_month column from created_at column
df_users['order_month'] = df_users['created_at'].dt.to_period('M')

# Create cohort column based on the first month of user_id activity
df_users['cohort'] = df_users.groupby('user_id')['created_at'].transform('min').dt.to_period('M') 
# Create another dataframe that contain number of unique numbers of customers for different order_month based on cohort
df_users_cohort = df_users.groupby(['cohort', 'order_month']).agg(n_customers=('user_id', 'nunique')).reset_index(drop=False)
print(df_users_cohort)

# Transform the df_cohort dataframe to cohort_pivot table
df_users_cohort['period_number'] = (df_users_cohort.order_month - df_users_cohort.cohort).apply(attrgetter('n'))
cohort_pivot_users = df_users_cohort.pivot_table(index = 'cohort', columns = 'period_number', values = 'n_customers')
#------------------------------------------------------------------------------------------------------------------------------------------

# Create cohort_pivot for df_gmv to get total gmv retentions
# Create order_month column from created_at column
df_gmv['order_month'] = df_gmv['created_at'].dt.to_period('M')

# Create cohort column based on the first month of user_id activity
df_gmv['cohort'] = df_gmv.groupby('user_id')['created_at'].transform('min').dt.to_period('M') 
# Create another dataframe that contain total gmv for different order_month based on cohort
df_gmv_cohort = df_gmv.groupby(['cohort', 'order_month']).agg(gmv=('grand_total', 'sum')).reset_index(drop=False)

print(df_gmv_cohort)
# Transform the df_cohort dataframe to cohort_pivot table
df_gmv_cohort['period_number'] = (df_gmv_cohort.order_month - df_gmv_cohort.cohort).apply(attrgetter('n'))
cohort_pivot_gmv = df_gmv_cohort.pivot_table(index = 'cohort',columns = 'period_number', values = 'gmv')
#---------------------------------------------------------------------------------------------------------------------------------------------
# Create cohort_pivot for df_orders to get total orders retentions
# Create order_month column from created_at column
df_orders['order_month'] = df_orders['created_at'].dt.to_period('M')

# Create cohort column based on the first month of user_id activity
df_orders['cohort'] = df_orders.groupby('user_id')['created_at'].transform('min').dt.to_period('M') 

# Create another dataframe that contain number of orders for different order_month based on cohort
df_orders_cohort = df_orders.groupby(['cohort', 'order_month']).agg(orders=('user_id', 'count')).reset_index(drop=False)
print(df_orders_cohort)
# Transform the df_cohort dataframe to cohort_pivot table
df_orders_cohort['period_number'] = (df_orders_cohort.order_month - df_orders_cohort.cohort).apply(attrgetter('n'))
cohort_pivot_orders = df_orders_cohort.pivot_table(index = 'cohort', columns = 'period_number', values = 'orders')







In [ ]:
# This is final cohort analysis image that you can copy and paste

#-----------------------------------------------------------------------------------------------------------------------------------------------
# CREATE COHORT ANALYSIS FOR ACTIVE BUYERS RETENTION
# Create retention matrix based on cohort_pivot dataframe, basically converting customer number to percentage
cohort_size_users = cohort_pivot_users.iloc[:,0]
retention_matrix_users = cohort_pivot_users.divide(cohort_size_users, axis = 0)

# Visualize the retention_matrix 
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(20, 12), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix_users, 
                mask=retention_matrix_users.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='YlGnBu', 
                ax=ax[1])
    ax[1].set_title('Active Users Retention: % of returned buyers over subsequent months', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_users = pd.DataFrame(cohort_size_users).rename(columns={0: 'Users'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_users, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap="Purples", 
                ax=ax[0])

    fig.tight_layout()

    fig.savefig('CA_users.png')

#-----------------------------------------------------------------------------------------------------------------------------------------------
# CREATE COHORT ANALYSIS FOR ORDERS RETENTION
# Create retention matrix based on cohort_pivot dataframe, basically converting customer number to percentage
cohort_size_orders = cohort_pivot_orders.iloc[:,0]
retention_matrix_orders = cohort_pivot_orders.divide(cohort_size_orders, axis = 0)

# Visualize the retention_matrix 
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(20, 12), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix_orders, 
                mask=retention_matrix_orders.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='YlGnBu', 
                ax=ax[1])
    ax[1].set_title('Orders Retention', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_orders = pd.DataFrame(cohort_size_orders).rename(columns={0: 'Orders'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_orders, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap="Purples", 
                ax=ax[0])

    fig.tight_layout()

    fig.savefig('CA_orders.png')


#-----------------------------------------------------------------------------------------------------------------------------------------------
# CREATE COHORT ANALYSIS FOR GMV RETENTION
# Create retention matrix based on cohort_pivot dataframe, basically converting customer number to percentage
cohort_size_gmv = cohort_pivot_gmv.iloc[:,0]
retention_matrix_gmv = cohort_pivot_gmv.divide(cohort_size_gmv, axis = 0)

# Visualize the retention_matrix 
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(20, 12), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix_gmv, 
                mask=retention_matrix_gmv.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='YlGnBu', 
                ax=ax[1])
    ax[1].set_title('GMV Retention', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # cohort size
    cohort_size_gmv = pd.DataFrame(cohort_size_gmv).rename(columns={0: 'GMV $'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(cohort_size_gmv, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap="Purples", 
                ax=ax[0])

    fig.tight_layout()

    fig.savefig('CA_gmv.png')



In [ ]:
# Connect to Google Sheets
from google.colab import auth
auth.authenticate_user()
! pip3 install --quiet gspread==3.6.0
from oauth2client.client import GoogleCredentials
import gspread

gc = gspread.authorize(GoogleCredentials.get_application_default())

#------------------------------------------------------------------------------------------------------------------------
# Export users cohort analysis

retention_matrix_users_flattened = pd.DataFrame(cohort_pivot_users.to_records())

# Convert cohort data type from Period to string. This is because datetime format is not JSON serialization which is unable to be uploaded to Google Sheets
retention_matrix_users_flattened['cohort']=retention_matrix_users_flattened['cohort'].astype(str)

# cohort_pivot['cohort'].astype(str)
retention_matrix_users_flattened.fillna('', inplace=True)

spreadsheet = gc.open("marketplace metrics")
current_worksheet = spreadsheet.worksheet('CA_users')
current_worksheet.update([retention_matrix_users_flattened.columns.values.tolist()] + retention_matrix_users_flattened.values.tolist())

#------------------------------------------------------------------------------------------------------------------------
# Export orders cohort analysis
retention_matrix_orders_flattened = pd.DataFrame(cohort_pivot_orders.to_records())

# Convert cohort data type from Period to string. This is because datetime format is not JSON serialization which is unable to be uploaded to Google Sheets
retention_matrix_orders_flattened['cohort']=retention_matrix_orders_flattened['cohort'].astype(str)

# cohort_pivot['cohort'].astype(str)
retention_matrix_orders_flattened.fillna('', inplace=True)

spreadsheet = gc.open("marketplace metrics")
current_worksheet = spreadsheet.worksheet('CA_orders')
current_worksheet.update([retention_matrix_orders_flattened.columns.values.tolist()] + retention_matrix_orders_flattened.values.tolist())

#------------------------------------------------------------------------------------------------------------------------
# Export gmv cohort analysis

retention_matrix_gmv_flattened = pd.DataFrame(cohort_pivot_gmv.to_records())

# Convert cohort data type from Period to string. This is because datetime format is not JSON serialization which is unable to be uploaded to Google Sheets
retention_matrix_gmv_flattened['cohort']=retention_matrix_gmv_flattened['cohort'].astype(str)

# cohort_pivot['cohort'].astype(str)
retention_matrix_gmv_flattened.fillna('', inplace=True)

spreadsheet = gc.open("marketplace metrics")
current_worksheet = spreadsheet.worksheet('CA_gmv')
current_worksheet.update([retention_matrix_gmv_flattened.columns.values.tolist()] + retention_matrix_gmv_flattened.values.tolist())


In [ ]:

# kill proxy by closing screen
!screen -X -S test quit

# This is the end of this notebook
